# LangGraph 

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random

In [ ]:
# Add some constants
nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody", "sparkly", "untrustworthy", "sarcastic", "squishy", "haunted"]

In [ ]:
load_dotenv(override=True)

In [ ]:
# Annotated type hint example
def shout(text: Annotated[str, "something to be shouted"]) -> str:
    print(text.upper())
    return text.upper()

shout("hello")

## 1. Define the State Object

In [ ]:
class State(BaseModel):
    messages: Annotated[list, add_messages]

## 2. Start the Graph Builder

In [ ]:
graph_builder = StateGraph(State) # class is passed, not an object

## 3. Create a Node
Nodes can be any Python function. The reducer, that we've set before (`add_messages`) gets automatically called to combine this response with previous responses.

In [ ]:
def our_first_node(old_state: State) -> State:
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]
    
    new_state = State(messages=messages)
    return new_state

graph_builder.add_node("first_node", our_first_node)

## 4. Create Edges

In [ ]:
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

## 5. Compile the Graph

In [ ]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

## Testing time

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

# 2nd example
Hope it made the point that LangGraph is all about Python functions and it does not need to involve LLMs.
Let's do the same, but with LLM.

In [ ]:
# Step 1 - define State object
class State(BaseModel): 
    messages: Annotated[list, add_messages]
    
# Step 2 - start the Graph Builder with State class
graph_builder = StateGraph(State)

# Step 3 - Create a Node
llm = ChatOpenAI(model='gpt-4o-mini')

def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

# Step 4 - Create Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Step 5 - Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)
    print(result)
    return result['messages'][-1].content

gr.ChatInterface(chat, type="messages").launch()